In [1]:
import sys
# add path to jupyter notebook
sys.path.append("/usr/local/lib")
import matplotlib.pyplot as plt          
import pyrealsense2 as rs
import numpy as np
import cv2
import torch
import torchvision
from jetbot import Robot
import jetbot
robot = Robot()
print(jetbot.__file__)

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)
    
model.load_state_dict(torch.load('best_model_cones.pth'))
device = torch.device('cuda')
model = model.to(device)



mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)


/usr/local/lib/python3.6/dist-packages/jetbot-0.3.0-py3.6.egg/jetbot/__init__.py


In [2]:

def get_np_frame(pipeline):
    try:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        
        colorizer = rs.colorizer()
        spatial = rs.spatial_filter()
        spatial.set_option(rs.option.holes_fill, 3)
        filtered_depth = spatial.process(depth_frame)
        colorized_depth = np.asanyarray(colorizer.colorize(filtered_depth).get_data())

        
        return colorized_depth
       
    
    except Exception as e:
        print("get_depth_frame >>> [ERROR] %s"%e)
#         n = np.zeros((224,224))
#         return n
    

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = x.astype(np.float32)
    x = cv2.resize(x,(224,224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x



In [3]:
import glob
import os
from PIL import Image, ImageFont, ImageDraw

font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 20)
font_probs = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 14)
path_to_img_folder = 'images'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
# try:
#     os.makedirs(path_to_img_folder)
# except FileExistsError:
#     print('Directory not created becasue they already exist')

def save_frames_with_telemetry(raw_image,current_probs,exploration,action,n_frames_stuck):
    prob_free,prob_left,prob_right,prob_blocked = current_probs
    # save image with telemetry for performance analysis (On-screen display data is for for 224x224 frame size) 
    img_file_nm = path_to_img_folder + '/img' + str(int(100.*time.time())) + '.jpeg'
    img = Image.fromarray(raw_image)
    
    draw = ImageDraw.Draw(img)
    draw.text((5,0),  "F", (255,255,0), font=font)
    draw.text((5,20), "L", (255,255,0), font=font)
    draw.text((5,40), "R", (255,255,0), font=font)
    draw.text((5,60), "B", (255,255,0), font=font)
    
    def fill_color(prob,threshold,below_color=(0,255,0,255),above_color=(255,0,0,255)):
        color = below_color
        if prob > threshold:
            color = above_color
        return color
    
    draw.rectangle([20,5,  int(20+80*prob_free) ,  18], fill=fill_color(prob_free, 0.5), outline=None)
    draw.rectangle([20,25, int(20+80*prob_left) ,  38], fill=fill_color(prob_left, 0.5), outline=None)
    draw.rectangle([20,45, int(20+80*prob_right),  58], fill=fill_color(prob_right, 0.5), outline=None)
    draw.rectangle([20,65, int(20+80*prob_blocked),78], fill=fill_color(prob_blocked, 0.5), outline=None)

    draw.text((22,4),  str(round(prob_free,2)), (0,0,0), font=font_probs)
    draw.text((22,24), str(round(prob_left,2)), (0,0,0), font=font_probs)
    draw.text((22,44), str(round(prob_right,2)), (0,0,0), font=font_probs)
    draw.text((22,64), str(round(prob_blocked,2)), (0,0,0), font=font_probs)
    
    # display frame number
    draw.text((5,84), 'FRAME #' + str(frame_counter), (0,0,0), font=font_probs)

    # display state of n_frames_stuck counter
    if exploration:
        draw.rectangle([5,104,170,119], fill=(255,0,0,255), outline=None)
    draw.text((5,104), 'FRAMES STUCK: #' + str(n_frames_stuck), (0,0,0), font=font_probs)

    action_displayed = action[0] + " " + str(action[1])
    if (action[0] == "FWRD"):
        action_displayed = action[0] + " " + str(action[1])
        draw.text((105,4), action_displayed, (255,0,0), font=font_probs)
    if (action[0] == "LEFT"):
        draw.text((105,24), action_displayed, (255,0,0), font=font_probs)
    if (action[0] == "RGHT"):
        draw.text((105,44), action_displayed, (255,0,0), font=font_probs)
    
    draw = ImageDraw.Draw(img)
    img.save(img_file_nm)    

In [4]:
import torch.nn.functional as F
import time

# Simple PD controller (Kp - proportional term, Kd - derivative term)
Kp = 0.18
Kd = 0.05

frwd_value = 0.2                      # Default value to drive forward (0 = no action, 1 = full motor capacity)
rot_value_when_exploring = 0.5        # Default value to spin to the right when robot is in exploration mode (0 = no action, 1 = full motor capacity)
min_prob_free_to_drive_frwd = 0.25    # Min probability prob_free for robot to drive forward 
max_n_frames_stuck = 10               # Limit on the number of frames the robot is stuck for. Once this limit is reached, robot goes into exploration mode (makes large right turn)
frame_counter = 0                     # Frame counter 
n_frames_stuck = 0                    # Initialize counter of the number of successive frames the robot is stuck for
exploration = False                   # Initialize binary variable which determines if robot is in exploration mode (when True.) Used to mark the related frames with red background  
data_log = []                         # Initialize the array whcih will store a history of telemetry readings and robot actions (for analysis and tuning)
recent_detections = []                 # Initialize the array to store the last frame data

def update(x):
    global robot, frame_counter, n_frames_stuck, exploration
#     x = change['new'] 
#     print("preprocessing...")
    x_ = preprocess(x)
#     print("read model...")
    y = model(x_)
    
    # apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    y = y.flatten()
   
    # extract probabilities of blocked, free, left and right
    prob_blocked = float(y[0])
    prob_free = float(y[1])
    prob_left = float(y[2])
    prob_right = float(y[3])
 
    # update list of recent detections
    while (len(recent_detections) >= 2):
        recent_detections.pop(0)
    recent_detections.append([prob_free,prob_left,prob_right,prob_blocked])
    
    # check if robot got stuck and update n_frames_stuck counter
    
    if prob_free < min_prob_free_to_drive_frwd:
        n_frames_stuck = n_frames_stuck + 1 
    else:
        n_frames_stuck = 0
        
    # calculate errors at times t (current) and t-1 (prev)    
    # error(t) and error(t-1): prob_left-prob_right   
    if len(recent_detections) == 2:
        current_probs = recent_detections[1]
        prev_probs = recent_detections[0]
    else:
        current_probs = [prob_free,prob_left,prob_right,prob_blocked]
        prev_probs = current_probs
                
    # error = prob_left-prob_right        
    current_error = current_probs[1] - current_probs[2]
    prev_error    = prev_probs[1] - prev_probs[2]

    # increment frame counter 
    frame_counter = frame_counter + 1
    
    # define functions which deterine (and return) robot actions
    def forward(value):
        robot.forward(value)
        
        return ("FWRD",round(value,2))

    def left(value):
        robot.left(value)

        return ("LEFT",round(value,2))

    def right(value):
        robot.right(value)

        return ("RGHT",round(value,2))
    
    action = ""
  
    # estimate rotational value to turn left (if negative) or right (if positive)
    # 0 = no action, 1 = full motor capacity)
    rot_value = - Kp * current_error - Kd * (current_error - prev_error)
    
    # store propotional and differential controller components for frame-by-frame analysis
    p_component = - Kp * current_error
    d_component = - Kd * (current_error - prev_error)
    
    # initalize binary flag showinf if robot rotates 
    robot_rotates = False
    
    # action logic
    # moving forward if there is no obstacles
    if prob_free > min_prob_free_to_drive_frwd:
        action = forward(frwd_value)
#         print(">>> forward")

    # turn left or right if robot is not blocked for a long time
    elif n_frames_stuck < max_n_frames_stuck:
        robot_rotates = True
        if rot_value < 0.0:
            action = left(-rot_value)
#             print(">>> left")
        else:
            action = right(rot_value)
#             print(">>> right")

    # activate exploration mode - robot turns right by a large (45-90 degree) angle if it failed to move forward for [max_n_frames_stuck] recent frames
    else:
        exploration = True
        robot_rotates = True
        action = right(rot_value_when_exploring)
#         time.sleep(0.1)
        n_frames_stuck = 0
    
#     time.sleep(0.001)
    
    # save frames - images with telemetry and robot actions data 
    save_frames_with_telemetry(x,current_probs,exploration,action,n_frames_stuck)
      
    # append frame's telemetry and robot action to the stored data 
    if not robot_rotates:
        rot_value = 0.
        p_component = 0.
        d_component = 0.
    if robot_rotates and exploration:
        rot_value = rot_value_when_exploring
        p_component = 0.
        d_component = 0.
    
    last_frame_data = [frame_counter, round(prob_free,3), round(prob_left,3), round(prob_right,3), round(prob_blocked,3),
                           action[0], action[1], round(rot_value,3), round(p_component,3), round(d_component,3), n_frames_stuck, exploration]
    data_log.append(last_frame_data)
    
    # reset variables
    exploration = False
    robot_rotates = False
    return {"status":"success"}

In [5]:
def main():
    try:
    
        # Configure depth and color streams
        pipe = rs.pipeline()
        config = rs.config()
        # config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
#         config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
        pipe.start(config)

        for x in range(5):
          pipe.wait_for_frames()
        
        print("initializaiton...")
        
        while True:
            
            x = get_np_frame(pipe) #np_array in frame
            res = update(x)
            
    
    
#     except Exception as e:
#         print("[ERROR] %s"%e)
        
    finally:
    
        pipe.stop()
    # Configure depth and color streams
    

    
main()

initializaiton...
>>> right
>>> right
>>> right
>>> right
>>> left
>>> right
>>> right
>>> right
>>> right
>>> right
>>> right
>>> right
>>> left
>>> right
>>> right
>>> right
>>> left
>>> left
>>> right
>>> right
>>> left
>>> right
>>> right
>>> right
>>> right
>>> left
>>> right
>>> left
>>> right
>>> right
>>> right
>>> right
>>> right
>>> left
>>> right
>>> right
>>> right
>>> right
>>> right
>>> right
>>> right
>>> left
>>> right
>>> left
>>> right
>>> right
>>> left
>>> right
>>> right
>>> right
>>> left
>>> left
>>> right
>>> left
>>> right
>>> left
>>> right
>>> right
>>> left
>>> right
>>> right
>>> right
>>> left


KeyboardInterrupt: 

In [7]:
robot.stop()
rs.pipeline().stop()

RuntimeError: stop() cannot be called before start()